In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura ainda não divulgou uma nova edição da Imersão IA com Google Gemini. A última edição ocorreu em fevereiro de 2024.

A melhor forma de saber sobre futuras edições é:

*   **Acompanhar a Alura:** Fique de olho no site oficial da Alura, nas redes sociais (YouTube, Instagram, LinkedIn, etc.) e na newsletter.
*   **Comunidade da Alura:** Participe da comunidade da Alura para ficar por dentro das novidades e eventos.
*   **Google:** Acompanhe os canais de comunicação do Google Developers Brasil.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: alura.com.br, tecmundo.com.br



In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Idades --- #
##########################################
def agente_idade(idade_usuario):

    idade = Agent(
        name="agente_idade",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente que irá pergutar pra a Idade da pessoa que está fazendo a pergunta. depois armazenar a idade.
        """,
        description="Agente que pergunta a Idade",
    )

    entrada_do_agente_idade = f"Idade: {idade}\n"

    lancamento_idade = call_agent(idade, entrada_do_agente_idade)
    return lancamento_idade

In [ ]:
################################################
# --- Agente 2: interesses --- #
################################################
def agente_interesses(idade_usuario, lancamentos_idades, interesse_pessoal):
    interesses = Agent(
        name="agente_interesses",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente interesses #################################################
        instruction="""
        Você é um assistente que utilizando a (lancamentos_idades) para perguntar qual o interesse da pessoa que está fazendo a pergunta.
        Você pode utilizar ferramenta de busca do Google (google_search) para fazer as perguntas de acordo com a idade da pessoa. guarde o interesse em uma variável.
        """,
        description="Agente que pergunta o interesse da pessoa",
        tools=[google_search]
    )

    # Incluir o interesse_pessoal na entrada do agente
    entrada_do_agente_interesses = f"Idade:{idade_usuario}\nLançamentos buscados: {lancamentos_idades}\nInteresse pessoal: {interesse_pessoal}"
    # Executa o agente
    # Chamar call_agent com apenas 2 argumentos, incorporando interesse_pessoal na mensagem
    plano_interesses = call_agent(interesses, entrada_do_agente_interesses)
    return plano_interesses

In [ ]:
######################################
# --- Agente 3: calculo --- #
######################################
def agente_calculo(Conta): # O parâmetro da função é 'Conta'
    calculo = Agent(
        name="agente_calculo",
        model="gemini-2.0-flash",
        instruction="""
            Você é um matemático. Sua tarefa é receber uma expressão matemática (representada pela variável 'Conta')
            e retornar APENAS o resultado numérico dessa expressão.
            Por exemplo, se a entrada for '2+2', sua saída deve ser exclusivamente '4'.
            Não adicione nenhuma palavra, explicação ou formatação extra.
            """,
        description="Agente que calcula contas matemáticas e retorna apenas o resultado."
    )

    entrada_do_agente_calculo = f"Por favor, calcule a seguinte expressão matemática: {Conta}"

    # Executa o agente
    rascunho_resultado_string = call_agent(calculo, entrada_do_agente_calculo)

    print(f"DEBUG [Agente 3]: Resultado bruto do agente de cálculo: '{rascunho_resultado_string}'")

    # É uma boa prática limpar a saída do LLM, especialmente se você espera apenas um número.
    if rascunho_resultado_string:
        resultado_limpo = rascunho_resultado_string.strip()
        return resultado_limpo
    else:
        # Lidar com caso de resposta vazia ou inesperada
        print(f"AVISO [Agente 3]: Agente de cálculo retornou uma resposta vazia para a conta: {Conta}")
        return None # Ou pode retornar "Erro no cálculo" ou levantar uma exceção

In [54]:
#########################################
# --- Agente 4: Revisor Conta --- #
##########################################
def agente_revisor_conta(idade_da_pessoa, descricao_dos_interesses, resultado_da_operacao_matematica):
    """
    Este agente recebe a idade do usuário, uma descrição dos seus interesses,
    e o resultado de uma conta matemática. Ele então explica o resultado
    de forma personalizada e didática.
    """
    nome_modelo_valido = "gemini-2.0-flash"

    instrucao_para_o_agente = f"""
        Você é um professor de matemática, criativo e empático.
        Suas resposta devem ser simples.
        relacione o número com o interesse da pessoa.
        Sua principal tarefa é analisar um resultado de uma operação matemática ({resultado_da_operacao_matematica}).
        Considerando a idade da pessoa ({idade_da_pessoa} anos) e seus principais interesses ({descricao_dos_interesses}),
        você deve elaborar uma explicação clara, envolvente e educativa sobre o resultado da conta.

        Use os interesses fornecidos como base para criar analogias, exemplos práticos ou
        contextualizações que tornem a matemática mais acessível e fascinante para essa pessoa.
        Por exemplo, se o interesse for 'exploração espacial' e a conta for '100 / 4 = 25',
        você poderia explicar que, se uma missão espacial durasse 100 dias e fosse dividida
        em 4 fases iguais, cada fase teria 25 dias.

        Se julgar necessário, utilize a ferramenta de busca do Google (Google Search)
        para obter informações, fatos curiosos ou exemplos que possam enriquecer sua explicação,
        conectando de forma inteligente a matemática aos interesses da pessoa.

        Sua resposta deve ser amigável, precisa e encorajadora.
        O objetivo é fazer a pessoa entender o resultado matemático e se sentir mais confiante com a matemática!
        você deve utilizar bastante emotis para demonstrar as explicações
        tente ser sempre muito visual e criativo nas respostas patra ajudar aqueles que tem dificuldade com a matemática.
        tente sempre usar emote mostrando o resultado
    """

    agente_explicador = Agent(
        name="agente_explicador_matematico",
        model=nome_modelo_valido,
        instruction=instrucao_para_o_agente,
        description="Agente especialista em explicar resultados matemáticos de forma personalizada, baseando-se na idade e nos interesses do usuário.",
        tools=[google_search]
    )

    input_para_o_agente = "Por favor, gere a explicação personalizada com base nas informações fornecidas em sua instrução."

    explicacao_personalizada = call_agent(agente_explicador, input_para_o_agente)

    return explicacao_personalizada

In [55]:
# --- Seu fluxo principal (parte relevante) ---
print("🚀 Iniciando o Sistema para responder questões matemáticas 🚀")

 # --- Obter o Tópico do Usuário ---
idade_usuario = input("❓ Por favor, digite a sua idade: ")

if not idade_usuario:
  print("Você esqueceu de digitar a idade!")
else:
  print(f"Maravilha! Sua idade é {idade_usuario}. Vamos continuar.")

lancamentos_relacionados_idade = agente_idade(idade_usuario)
print("\n--- 📝 Resultado do Agente 1 ---\n")
display(to_markdown(lancamentos_relacionados_idade))
print("--------------------------------------------------------------")

interesse_pessoal = input("O que você mais gosta?")

plano_de_interesses = agente_interesses(idade_usuario, lancamentos_relacionados_idade, interesse_pessoal)
print("\n--- 📝 Resultado do Agente 2 ---\n")
display(to_markdown(plano_de_interesses))
print("--------------------------------------------------------------")



conta_digitada = input("Digite a conta matemática que você está em dúvida (ex: 2+2, 15*4): ")

resultado_da_conta_calculado = agente_calculo(conta_digitada)
print("\n--- 📝 Resultado do Agente 3 ---\n")
display(to_markdown(str(resultado_da_conta_calculado)))
print("--------------------------------------------------------------")


explicacao_final = agente_revisor_conta(idade_usuario, plano_de_interesses, str(resultado_da_conta_calculado))
print("\n--- 📝 Resultado do Agente 4 (Explicação Personalizada) ---\n")
display(to_markdown(explicacao_final))
print("--------------------------------------------------------------")

🚀 Iniciando o Sistema para responder questões matemáticas 🚀
❓ Por favor, digite a sua idade: 20
Maravilha! Sua idade é 20. Vamos continuar.

--- 📝 Resultado do Agente 1 ---



> Olá! Para começar, qual é a sua idade?
> 


--------------------------------------------------------------
O que você mais gosta?cachorro

--- 📝 Resultado do Agente 2 ---



> Olá! Que bom saber que você tem 20 anos e gosta de cachorros!
> 
> 
> Que legal que você gosta de cachorros! Já pensou em qual tipo de cachorro você gostaria de ter? Por exemplo, você prefere um filhote ou um cachorro adulto? E você mora em casa ou apartamento? Isso pode influenciar na escolha da raça, pois algumas se adaptam melhor a espaços menores.
> 


--------------------------------------------------------------
Digite a conta matemática que você está em dúvida (ex: 2+2, 15*4): 23*2
DEBUG [Agente 3]: Resultado bruto do agente de cálculo: '46

'

--- 📝 Resultado do Agente 3 ---



> 46

--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Explicação Personalizada) ---



> Olá! Que bom saber que você tem 20 anos e gosta de cachorros! 🐶 🎉
> 
> Vamos explorar o número 46 de uma forma super divertida e relacionada ao seu amor por cachorros! 🐾
> 
> **Visualizando o 46:** Imagine que você tem 46 petiscos deliciosos para dar aos seus cachorros. 🍖😋
> 
> **Dividindo para Compartilhar:** Se você tem vários cachorros (ou planeja ter! 🐕‍🦺🐩), pode dividir esses 46 petiscos entre eles. Por exemplo:
> 
> *   Se você tiver 2 cachorros 🐕🐕, cada um receberá 23 petiscos! (46 ÷ 2 = 23) 🎁🎁
> *   Se você tiver 4 cachorros 🐕🐕🐕🐕, cada um receberá 11 petiscos, e sobrarão 2! (46 ÷ 4 = 11, resto 2) 🎁🎁🎁🎁
> 
> **Anos Caninos:** Sabia que a idade dos cachorros é diferente da nossa? 🤔 Embora não exista uma fórmula exata, uma estimativa comum é que 1 ano humano equivale a 7 anos caninos. Então, 46 anos "caninos" seriam aproximadamente 6,5 anos humanos! 🎉
> 
> **Curiosidade:** Existem mais de 340 raças de cachorros reconhecidas mundialmente! 🐕🌍
> 
> **Conclusão:** O número 46 pode parecer simples, mas, assim como os cachorros, ele é cheio de possibilidades e surpresas! 🤩 Continue explorando a matemática e o mundo canino! 🐕‍🦺🐾🔢


--------------------------------------------------------------
